actually this nb is outdated. better if you use scrape_pqs_save_only in conjunction with clean_scraped_hansard.

run the other notebooks. should give two csv files. this one will combine them and clean up the data too.

In [1]:
from bs4 import BeautifulSoup as bs
import ast
import os
import os.path
from datetime import datetime
from enum import Enum
import re
import pandas as pd
import numpy as np
import datetime


**for merging with mps.csv**

In [2]:
mp_df = pd.read_csv('mps.csv')
mp_df.Party = mp_df.Party.apply(ast.literal_eval)
mp_df.Parliaments = mp_df.Parliaments.apply(ast.literal_eval)
mps = dict(
    zip(mp_df.Name.apply(lambda x: x.replace('.', '').replace(',', '').lower()), # keys
    zip(mp_df.Name, mp_df.Party, mp_df.Parliaments))) # values
mp_names = list(mps.keys())

In [3]:
alr_matched = set() # honorific+names that have alr been matched to names so we don't spam the print
ministers_found = set() # minister titles that have alr been found (to be used for future searches in case of typos)

In [4]:
# for matching honorific+name in report to actual mp data.
# cannot simply remove honorific as the programmer doesn't have an exhaustive list
# of honorifics, and some are quite rare in everyday use (e.g. Inche Rahamat Bin Kenap).
def honorific_name_to_mp_data(honorific_name):
    honorific_name = honorific_name.replace('.','').replace(',','').lower().strip()
    
    # try the easy way first (find and remove honorific)
    honorific_match = re.match(r'(mr|mrs|ms|miss|mdm|dr|er dr|prof|assoc prof|er|asst prof|assoc prof dr|inche|encik)', honorific_name)
    if honorific_match:
        name = honorific_name[honorific_match.span()[1]+1:]
        if name in mps.keys():
            return mps[name]
        
        # seems quite common for them to write "asked" twice in the hansard proceedings
        last_asked = name.rfind(' asked')
        if last_asked and name[:last_asked] in mps.keys():
            return mps[name[:last_asked]]

        # slightly harder way (rearranging words)
        for mp_name in mp_names:
            mp_name_words = set(mp_name.split(' '))
            name_words = set(name.split(' '))
            if mp_name_words == name_words:
                if (honorific_name, mp_name) not in alr_matched:
                    #print(f'rearranging matched {honorific_name} to {mp_name}')
                    alr_matched.add((honorific_name, mp_name))
                return mps[mp_name]
            
            # for omission of chinese name
            if len(mp_name_words) - len(name_words) <= 2 and len(name_words) >= 2 and name_words.issubset(mp_name_words):
                if (honorific_name, mp_name) not in alr_matched:
                    print(f'allowing omitted words in name matched {honorific_name} to {mp_name}')
                    alr_matched.add((honorific_name, mp_name))
                return mps[mp_name]
                
    digit_match = re.search('\d+', honorific_name)
    if digit_match:
        # names shldn't have digits
        honorific_name = honorific_name[digit_match.span()[1]:]
        return honorific_name_to_mp_data(honorific_name)
        
    # the hard way (levenshtein)
    min_levenshtein = 99999
    min_ind = -1
    for i in range(len(mp_names)):
        l_dist = levenshtein(mp_names[i], honorific_name)
        if l_dist < min_levenshtein:
            min_levenshtein = l_dist
            min_ind = i
            
    if (honorific_name, mp_names[min_ind]) not in alr_matched:
        print(f'levenshtein matched {honorific_name} to {mp_names[min_ind]}')
        alr_matched.add((honorific_name, mp_names[min_ind]))
    return mps[mp_names[min_ind]]


# borrowed from: https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/
# we use levenshtein as it helps to protect against typos too, like the "asked asked" in:
# https://sprs.parl.gov.sg/search/sprs3topic?reportid=oral-answer-2822
def levenshtein(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

In [5]:
pqs = []

we assume that all pqs are prefaced with #. (non sprs) or # (sprs). ignore follow up qns since we are only interested in mapping mps to topics, and the follow up qns will always be from the same mp and on the same topic.

notes regarding minister titles:
* Minister for Culture, Community and Youth is the only minister title with a comma
* but there used to be Minister for Information, Communication and the Arts and Minister for Community Development, Youth and Sports
* no questions were ever directed to minister mentor

In [6]:
cap_word = r'[A-Z][a-z]+'
cap_words = f'({cap_word})( {cap_word})*'
mccy = '(Acting )?Minister for Culture, Community and Youth'
mica = '(Acting )?Minister for Information, Communications and the Arts'
mcdys = '(Acting )?Minister for Community Development, Youth( and|,) Sports'
micma = 'Minister-in-charge of Muslim Affairs'
minister_for_something = f'({cap_words} )?Minister (for|of) (the )?{cap_words}( and (the )?{cap_words})?'
something_minister = f'{cap_words} Minister'
one_minister_regex = f'(({mccy})|({mica})|({mcdys})|({micma})|({minister_for_something})|({something_minister}))'
minister_regex = re.compile(f'{one_minister_regex}( and (the )?{one_minister_regex})?') # can have multiple targets

def first_two_capitalized(words):
    return words[0][0].isupper() and words[1][0].isupper() and words[0][1] and words[0][1].islower()

def trim_off_non_pq_content_at_start(para):
    para_words = para.split(' ')
    if first_two_capitalized(para_words):
        return para
    
    # i assume honorific+name has at least two words capitalized and non-numbers
    while not first_two_capitalized(para_words):
        para_words = para_words[1:]
    
    return ' '.join(para_words)

def get_ministers_and_question(para):
    minister_match = re.search(minister_regex, para)
    if not minister_match:
        # report might've been in the wrong case; try to match to existing ministers
        minister_match = re.search(
            '(' + '|'.join(list(map(lambda s: s.lower().replace(' for ', '.{1,5}'), ministers_found))) + ')',
            para.lower()
        )
        if not minister_match:
            minister_match = re.search(
            '(' + '|'.join(list(map(lambda s: s.lower().replace(' for ', '(\s)*(for|of)(\s)*'), ministers_found))) + ')',
            para.lower()
        )
        if not minister_match:
            for existing_minister in ministers_found:
                if existing_minister.replace(' ', '') in para.replace(' ', ''):
                    minister_match = re.search('(\s)?'.join(c for c in existing_minister.replace(' ', '').lower()), para.lower())
                    break   
        min_levenshtein = 99999
        min_minister = None
        for existing_minister in ministers_found:
            dist = levenshtein(existing_minister, minister_match.group())
            if dist < min_levenshtein:
                min_levenshtein = dist
                min_minister = existing_minister
        
        print(f'found minister: {str(minister_match.group())}; matched to {min_minister}')
        askee = min_minister
    else:
        askee = para[:minister_match.span()[1]].replace(' of ', ' for ')
        ministers_found.add(askee)
    question = para[minister_match.span()[1]:].strip()
    if ' and Leader' in askee:
        askee = askee[:-11]
        print(f'removed "and leader" from {askee}')
    
    if not re.search('and (the )?Minister', askee):
        return (askee,), question   
    else:
        askee = askee.replace('and the Minister', 'and Minister')
        askees = askee.split(' and Minister')
        return (askees[0], 'Minister' + askees[1]), question

def pq_para_to_pq(pq_para):
    para = pq_para[0]
    para = re.sub(r'Page:\s+\d+', '', para)
    para = re.sub(r'\s+', ' ', para)
    para = trim_off_non_pq_content_at_start(para) # sometimes we end up mistaking other numbers in the text as being the pq numbers. so we deal w that here.
    asker_honorific_name, para = para.split(' asked the ', 1)
    
    ministers, question = get_ministers_and_question(para)
    if question[0] == ',':
        question = question[1:].strip()
    asker, asker_party, asker_parls = honorific_name_to_mp_data(asker_honorific_name.strip())
    pqs.append([asker, asker_party, asker_parls, ministers, question, *pq_para[1:]])


In [7]:
pq_paras = pd.read_csv('pq_paras.csv').values.tolist()

In [8]:
for pq_para in pq_paras:
    try:
        if pq_para[2] >= 12:
            pq_para_to_pq(pq_para)
        else:
            break
    except Exception as e:
        print(str(e))
        continue

allowing omitted words in name matched mr murali pillai to murali pillai sc
allowing omitted words in name matched mr alex yam to alex yam ziming
allowing omitted words in name matched mr mohd fahmi aliman to mohd fahmi bin aliman
found minister: minister for transport; matched to Minister for Transport
levenshtein matched mr yee chia hsing ? to yee chia hsing
allowing omitted words in name matched mr david ong to david ong kim huat
levenshtein matched mrs lina chiam to lina chiam
removed "and leader" from Minister for Defence
levenshtein matched assoc prof dr muhammad faishal ibrahim to muhammad faishal ibrahim
found minister: minister or national development; matched to Minister for National Development


In [9]:
pq_df = pd.DataFrame(pqs, columns=['asker_name', 'asker_party', 'asker_parliaments', 'askee', 'question', 'sitting_date', 'parliament_no', 'report_section'])
pq_df.sitting_date = pq_df.sitting_date.apply(lambda dt: dt if dt.find('00:00:00') == -1 else dt[:dt.find('00:00:00')-1])
pq_df.sitting_date = pq_df.sitting_date.apply(lambda dt: dt if dt.find('0:00') == -1 else dt[:dt.find('0:00')-1])
pq_df.asker_party = pq_df.asker_party.apply(lambda x: x[0])
pq_df

,asker_name,asker_party,asker_parliaments,askee,question,sitting_date,parliament_no,report_section
0,Pritam Singh,Workers' Party,"(12, 13, 14)","(Prime Minister,)",(a) what is the format of the Government's aft...,9/5/2022,14,Oral Answers to Questions
1,Seah Kian Peng,People's Action Party,"(11, 12, 13, 14)","(Minister for Law,)",(a) whether there were similar instances of ch...,9/5/2022,14,Oral Answers to Questions
2,He Ting Ru,Workers' Party,"(14,)","(Minister for Law,)",(a) whether any steps need to be taken to addr...,9/5/2022,14,Oral Answers to Questions
3,Hany Soh,People's Action Party,"(14,)","(Minister for Law,)",(a) whether the Singapore Institute of Legal E...,9/5/2022,14,Oral Answers to Questions
4,Dennis Tan Lip Fong,Workers' Party,"(13, 14)","(Minister for Transport,)",whether the fall in the volume of Singapore's ...,9/5/2022,14,Written Answers to Questions for Oral Answer N...
...,...,...,...,...,...,...,...,...
4941,Sylvia Lim,Workers' Party,"(11, 12, 13, 14)","(Deputy Prime Minister, Minister for Home Affa...",with regard to the fire at the Shell refinery ...,20/10/2011,12,Oral Answers to Questions
4942,Sylvia Lim,Workers' Party,"(11, 12, 13, 14)","(Minister for Trade and Industry,)",if he would provide an interim assessment on t...,20/10/2011,12,Oral Answers to Questions
4943,Christopher de Souza,People's Action Party,"(11, 12, 13, 14)","(Minister for Information, Communications and ...",(a) why MDA considers it better to have an opt...,20/10/2011,12,Written Answers to Questions
4944,Christopher de Souza,People's Action Party,"(11, 12, 13, 14)","(Deputy Prime Minister, Minister for Manpower)",whether the Ministry would consider calibratin...,20/10/2011,12,Oral Answers to Questions


In [10]:
pq_df.to_csv('pqs.csv', index=False)

In [12]:
assert all(pq_df.parliament_no < 15) and all(pq_df.parliament_no >= 12)
assert all(map(lambda x: not x[0].isupper(), pq_df.question.values)) 
assert all(map(lambda x: not x[:3] == 'and', pq_df.question.values)) 
print(set(pq_df.asker_party.values))
print()
print(set([askee for sublist in pq_df.askee for askee in sublist]))

{'', "People's Action Party", 'Progress Singapore Party', 'Nominated Member of Parliament', "Workers' Party", "Singapore People's Party"}

{'Deputy Prime Minister', 'Minister for Sustainability and the Environment', 'Minister for Defence', 'Minister for Culture, Community and Youth', 'Minister for Transport', 'Prime Minister', 'Minister for the Environment and Water Resources', 'Acting Minister for Community Development, Youth, Sports', 'Minister for Health', 'Minister for Home Affairs', 'Minister-in-charge for Muslim Affairs', 'Minister for Finance', 'Acting Minister for Social and Family Development', 'Minister for Education', 'Acting Minister for Culture, Community and Youth', 'Minister for National Development', 'Acting Minister for Education', 'Minister for Communications and Information', 'Minister for Information, Communications and the Arts', 'Minister for Social and Family Development', 'Minister for Law', 'Acting Minister for Manpower', 'Acting Minister for Community Developm

In [13]:
len(set(pq_df.asker_name.values))

129

In [14]:
min_count = 999
min_mp = None
max_count = 0
max_mp = None
less_than_ten = 0
for name in pq_df.asker_name.values:
    count_here = pq_df[pq_df.asker_name == name]['asker_name'].count()
    if count_here < min_count:
        min_count = count_here
        min_mp = name
    if count_here > max_count:
        max_count = count_here
        max_mp = name
    if count_here < 10:
        less_than_ten += 1
        
min_count, min_mp, max_count, max_mp, less_than_ten

(1, 'Poh Li San', 233, 'Gan Thiam Poh', 181)